In [165]:
import selenium
from selenium import webdriver as wb
import pandas as pd
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm
import numpy as np
pd.options.mode.chained_assignment = None

In [68]:
page = wb.Chrome('chromedriver.exe')
page.get('https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki?sort=popular&page=1')

C:\Users\5956~1\AppData\Local\Temp/ipykernel_2468/2649964490.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  page = wb.Chrome('chromedriver.exe')


In [70]:
def pageBottom(driver): # Функция, чтобы прокрутить страницу вниз
    bottom=False
    a=0
    while not bottom:
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {a});")
        if a > new_height:
            bottom=True
        a+=5
## Источник: https://stackoverflow.com/questions/30942041/slow-scrolling-down-the-page-using-selenium

def get_links(): #Собириаем ссылки на товары
    prod = page.find_elements(By.CLASS_NAME, "product-card__wrapper")
    print(len(prod))
    for el in prod:
        try:
            lst_of_links.add(el.find_element(By.TAG_NAME,'a').get_property('href'))
        except:
            continue

In [71]:
url = 'https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki?sort=popular&page='
lst_of_links = set()

for i in range(1, 18):
    link = url + str(i)
    page.get(link)
    pageBottom(page)
    get_links()
print (len(lst_of_links))
lst = pd.DataFrame(lst_of_links)
lst.to_csv('url.csv', sep = '\n', index = False,  header = False )

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
8
1608


Итак, код сработал исправно: на каждой из первых 16-ти страниц располагается ровно 100 товаров (далее необходимо кликнуть `Следующая страница`), и ровно столько ссылок на каждой странице и собрал код выше. Теперь они хранятся в файле `url.csv`)

In [75]:
laptops = {}
page.get('https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki?sort=popular&page=1')
with open('url.csv', 'r', encoding = 'utf-8') as file:
    for line in tqdm(file):
        url = line.strip()
        page.get(url)
        time.sleep(5)
        try:
            name = page.find_element(By.CLASS_NAME, 'product-page__header-wrap').find_element(By.TAG_NAME, 'h1').text
            price = page.find_element(By.CLASS_NAME, 'price-block__final-price').text
        
            try:
                page.find_elements(By.CLASS_NAME, 'collapsible__bottom')[-1].find_element(By.TAG_NAME, 'button').click()
            except: 
                continue
            d = dict()
            a = page.find_elements(By.CSS_SELECTOR, 'td.product-params__cell')
            b = page.find_elements(By.CSS_SELECTOR, 'th.product-params__cell')
            for i, j in zip(a, b):
                ch = j.text
                q = i.text
                d[ch]= q
            try:
                diagonal = d['Диагональ экрана']
            except:
                diagonal = ''
            try: 
                ssd = d['Объем накопителя SSD']
            except:
                ssd = ''
            try:
                cores = d['Количество ядер процессора']
            except:
                cores = ''
            try:
                compl = d['Комплектация']
            except:
                compl = ''
            try:
                ac = d['Емкость аккумулятора']
            except:
                ac = ''
            try:
                weight = d['Вес без упаковки (кг)']
            except:
                weight = ''
            try:
                op = d['Операционная система']
            except:
                op = ''
            try:
                ram = d['Объем оперативной памяти (Гб)']
            except:
                ram = ''
            laptops[name] = {'Цена': price,
                             'Диагональ экрана': diagonal, 
                             'Объем накопителя SSD': ssd,
                             'Объем оперативной памяти': ram,
                             'Количество ядер процессора':cores,
                             'Комплектация': compl,
                             'Емкость аккумулятора': ac,
                             'Вес': weight,
                             'Операционная система': op
                            }
        except:
            continue
df = pd.DataFrame(laptops).T
pd.DataFrame(laptops).T.to_csv('laptops.csv')
df

1608it [3:12:26,  7.18s/it]


,Цена,Диагональ экрана,Объем накопителя SSD,Объем оперативной памяти,Количество ядер процессора,Комплектация,Емкость аккумулятора,Вес,Операционная система
Ноутбук Megabook T1 (i5) 16/512Гб (Linux),43 230 ₽,"15.6""",512 Гб,16 ГБ,4,"документация, адаптер питания",70 Вт*Ч,1.48 кг,Linux
"Ноутбук ASUS VivoBook 15 15.6""FHD OLED I5/8Gb/512SSD/W11",75 800 ₽,15.6,512,8 ГБ,4,"Ноутбук, зарядное устройство, документация",,1.4 кг,Windows Home
"Ноутбук 15.6"" Notebook Pink Intel Celeron N5095 2.0GHz",32 679 ₽,15.6,512 Гб,16,,коробка; ноутбук; накладка на клавиатуру,,,
Ноутбук Lenovo Ideapad 3 Intel Core i3-1115G4,39 800 ₽,15.6,256 Гб,4,4,"Ноутбук, зарядное устройство, сетевой адаптер",,1.5 кг,Windows Pro
Карта роблокс Roblox 25 USD 2000 робукс Robux,2 450 ₽,,,,,Подарочная карта роблокс ребусы roblox robux к...,,,
...,...,...,...,...,...,...,...,...,...
"Ноутбук AZ-1511 15.6""/4x2.0Ghz/16Gb/256Gb",28 025 ₽,15.6'',256 Гб,16 ГБ,4,Блок питания - 1 шт; Ноутбук - 1 шт,5000 mAh,1.9 кг,Windows
"Ноутбук Lenovo Thinkbook 15 G2 ITL Intel Core i5 1135G7/8Gb/SSD256Gb/15.6""/IPS/FHD/Win11Pro/grey",61 725 ₽,15.6,256 Гб,8 ГБ,4,Ноутбук - 1 шт,,1.7 кг,WIN11 Pro
"Ноутбук Acer NX.ABDER.007(i5/16,1/256Гб/8Гб/UMA/noOS)",50 760 ₽,"16.1""",256 Гб,8 ГБ,4,,,,noOS
"Ноутбук Gigabyte AORUS 15P KD (15.6 "", FHD 1920x1080 (16:9), Intel, Core i7, 16 Гб, SSD)",205 000 ₽,"15.6 """,1 Тб,16 ГБ,8 шт; 8 Ядер 16 потоков,,6700 mAh,2.29 кг,Windows 11 Home


Получаем следующую таблицу. Данные выглядят очень разнородно: им предстоит длительная обработка. Вся она будет сделана в файле `Проект`. Сейчас лишь немного подправим:

In [87]:
df[df == ''] = np.nan # все пропуски заполним NaN

In [91]:
df = df.dropna (subset=['Объем накопителя SSD']) 
''' Удалим товары с отсутствующим Объемом SSD. Скорее всего, это не ноутбуки, а случайно 
попавшие на страницу иные товары (выше можно заметить, что такие товары действительно есть:
например, "Карта роблокс Roblox 25 USD 2000 робукс Robux")

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, Ноутбук Megabook T1 (i5) 16/512Гб (Linux) to Ноутбук IdeaPad 3 15IGL05 lenovo
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Цена                        1106 non-null   object
 1   Диагональ экрана            1104 non-null   object
 2   Объем накопителя SSD        1106 non-null   object
 3   Объем оперативной памяти    1103 non-null   object
 4   Количество ядер процессора  1082 non-null   object
 5   Комплектация                815 non-null    object
 6   Емкость аккумулятора        541 non-null    object
 7   Вес                         817 non-null    object
 8   Операционная система        1062 non-null   object
dtypes: object(9)
memory usage: 86.4+ KB


In [95]:
df.to_csv('data.csv')